<a href="https://colab.research.google.com/github/rsadaphule/jhu-dnn/blob/main/Team7%F0%9F%8F%86%F0%9F%8C%8CStellar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🌌Stellar</font>](https://www.kaggle.com/competitions/5jun23jh-stellar/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 5jun23jh-stellar        # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
lb =!kaggle competitions leaderboard --show                   # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 5jun23jh-stellar


In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression as LR
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA, LinearDiscriminantAnalysis as LDA
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 1.32 s, sys: 165 ms, total: 1.48 s
Wall time: 1.81 s


In [ ]:
df = pd.read_csv('XY_Stellar.csv'); df

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID,Class
0,11.64,21.28,26.28,26.15,24.05,18.87,19.00,8848,5,272,0.84,7740,56824,833,NaN
1,173.09,42.21,22.51,22.83,22.21,19.55,19.96,4156,3,486,0.81,9041,58067,428,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199998,131.31,44.27,24.07,24.64,21.63,19.20,19.03,7076,3,251,0.55,6014,56166,1021,G
199999,22.59,0.25,25.30,25.56,24.09,19.41,19.96,5164,4,511,1.26,9590,57969,878,G


In [ ]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   alpha     200000 non-null  float64
 1   delta     200000 non-null  float64
 2   u         200000 non-null  float64
 3   g         200000 non-null  float64
 4   r         200000 non-null  float64
 5   i         200000 non-null  float64
 6   z         200000 non-null  float64
 7   run_ID    200000 non-null  int64  
 8   cam_col   200000 non-null  int64  
 9   field_ID  200000 non-null  int64  
 10  redshift  200000 non-null  float64
 11  plate     200000 non-null  int64  
 12  MJD       200000 non-null  int64  
 13  fiber_ID  200000 non-null  int64  
 14  Class     160000 non-null  object 
dtypes: float64(8), int64(6), object(1)
memory usage: 22.9+ MB


In [ ]:
# Change string labels to numbers in order of increasing size of the entity (Star < Quasi Star < Galaxy)
# df.Class = df.Class.apply(lambda C: -1 if C=='S' else 0 if C=='Q' else 1 if C=='G' else None)

In [ ]:
vX = df.query('Class!=Class').drop('Class', axis=1)  # slice a test sample
tXY = df.query('Class==Class')                       # slice training sample
tX, tY = tXY.drop('Class', axis=1), tXY.Class        # split into training I/O

In [ ]:
def ScatterCorrHist(df):
  def corrdot(*args, **kwargs):
    # credit: https://stackoverflow.com/questions/48139899
    corr_r = args[0].corr(args[1], 'pearson')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca();
    ax.set_axis_off();
    msz = abs(corr_r) * 5000   # marker size
    fsz = abs(corr_r) * 40 + 5 # font size
    ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

  sns.set(style='white', font_scale=.8);
  g = sns.PairGrid(df, aspect=1, diag_sharey=False);
  g.fig.set_size_inches(20,10)
  g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
  g.map_diag(sns.histplot, kde_kws={'color':'black'});
  g.map_upper(corrdot);
  g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
  _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
  _ = plt.show();

# ScatterCorrHist(tXY.head(200))

In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.




## **Task 1. Preprocessing Pipeline**

**Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.**

- Our pre-processing pipeline relied heavily on feature engineering. The objective was to get a good mix of existing features and generated features to increase the score.
- Transformers were used to scale and transform existing features into new features.

**1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)**

Many of the attributes were identifiers (IDS); MJD is a *Modified Julian Date*; these values were removed: ['run_ID', 'cam_col', 'field_ID', 'MJD', 'fiber_ID'].

The target Variables are skewed. There are many more G *galaxies* than S *stars* or Q *Quasars combined.  It was necessary to keep track of how the preprocessing impacted the distribtion so as to not remove too many from one of the lower target counts.

Many of these features were found using brute force search. A collection of candidate operations was assembled and a baseline performance was computed on the unmodified features. Random operations were then chosen from that collection and applied to random features. The performance was then recomputed.

If performance improved, then that operation-feature pair was reported. This process was then repeated. We then sorted through these suggested features by hand and picked those that seemed simple and likely to be applicable.

One issue with a technique like this is its liability to overfit the training data if one blindly applies all of the suggested features. Performance on the training set will grow arbitrarily high as the number of features grows.

For this reason, we applied a number of safeguards against overfitting.
- Firstly, cross-validation was used and only improvements on the validation sets were reported.
- Secondly, the data was only processed in this way in 20,000 row chunks. When a very promising feature was reported, that feature was then tested against a different 20,000 row chunk of training data to confirm that the same increase in performance was observed.

Many times, testing against a different chunk did not improve and when this happened, the suggested engineered feature was thrown away. Only when substantial evidence was collected that an automatically generated feature would improve the performance of our model was that engineered feature finally adopted.

**2. How do you evaluate the effectiveness of these elements?**
- All components that could used a random seed, all used the same random seed. This was set at the top of the project. When *colabs* were exchanged between teammates, the scores would be identical as a result.
- As new features were generated, the performance was measured using five fold cross validation.
- Different models to include *logistic regression*, *SGDClassifier*, *QuadraticDiscriminantAnalysis*,  and *LinearDiscriminantAnalysis* were evaluated with LSA being selected as the model to use.


**3. What else have you tried that worked or didn't?**
- After deciding on *LDA* many combinations of variables were trialed that would inch the score up ever so slightly.
- As discussed earler a collection of Lambda functions were created to try different combinations. This was very helpful.
- MinMaxScaling worked well on a selction of features.
- A transform called *QuantileTransformer* worked well and increased the score.  Features were added based on scaling attributes using it.
- StandardScaler and RobustScaller did not work as well as MinMaxScaler.
- VarianceThreshold was used and did improve the processing speed significantly.
- Third order polynomials were tried but with so many features, the processing time exceeded the threshold permitted.
- Second order polynomials with many features worked well.
- Feature selection using *, SelectKBest*, worked well when trialing third order polynomials. With the second order polynomials, this was set to keep all the features. The pipeline was tried with and without it; with it the performance was much better than without it.
- Many combinations of features were trialed. All possibilities cannot be mentioned here but when the score increased the new feature was kept.


## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Explain your modeling approach**
Our modeling approch started after reviewing the data.
1. Feature engineering -- generate pleanty of features.
2. Eavaluate models and select the one that balanced performance measured in terms of the score and speed in terms of getting it done within the acceptable timeframe. Logistic Regression, SGDClassifier, QuadraticDiscriminantAnalysis, and LinearDiscriminantAnalysis (LDA) were all evaluated.
3. LDA selected and team worked on a generating and trialing different feature combinations to increase the score.

**1. How did these decisions guide you in modeling?**
The Score and getting it done in time were what drove feature and model selection.  Getting a good combination of features was essential to completing on time.

**2. How do you evaluate the effectiveness of these elements?**
In trialing new things 5 fold cross validation was used in some cases. While in others only a single score was used.

**3. What else have you tried that worked or didn't?**
Many of the other models were tried but did not perform as well as LDA. Hyperparameter selection was performed on those and it proved very time consuming.  LDA was simpler and had few hyperparameters to adapt.

**Below is a work of Team 7.**
- Kristina Moralic
- Thomas Marshall
- Shawn Simon
- Anthony Sheller


In [ ]:
%%time


from copy import deepcopy
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,VarianceThreshold
import warnings
#from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler

#import sklearn
#sklearn.set_config(display="diagram"),

# Initalize Random State used by all that use a random state
RANDOM_STATE = 5
def shuffle_and_split_data(data, test_ratio,r_int=2):
    '''
    Shuffle and split accordingly.

    The return value will be stratified as the data is imbalanced

    '''
    data = data.sample(frac=1,random_state = RANDOM_STATE).reset_index(drop=True)
    train, test = train_test_split(data, test_size=test_ratio, stratify=df['Class'],random_state=RANDOM_STATE)
    return train, test

# A dictionary of useful lambda functions for creating new features
ops2 = {
 'log': lambda x: np.log(np.maximum(0, x) + 1),
 'exp': lambda x: np.exp(x),
 'sig': lambda x: 1 / (1 + np.exp(-x)),
 'si+': lambda x: x + 1 / (1 + np.exp(-x)),
 'mul': lambda x, y: x * y,
 'mul3': lambda x, y, z: x * y * z,
 'logadd': lambda x, y: np.log(np.maximum(0, x + y) + 1),
 'expadd': lambda x, y: np.exp(x + y),
 'norm': lambda x: (x - x.min()) / (x.max() - x.min()),
 'zsc': lambda x: (x - x.median()) / (x.std()),
 'div': lambda x, y: x / y,
 'asin': lambda x: np.arcsin(x),
 'acos': lambda x: np.arccos(x),
 'atan': lambda x: np.arctan(x),
 'tanh': lambda x: np.tanh(x),
 'drop': None
 }

# Drop the features not useful in our pipeline
df = df.drop(['run_ID', 'cam_col', 'field_ID', 'MJD', 'fiber_ID'], axis=1)


# Preprocessing -- create new features
df['logred'] = ops2['log'](df.redshift)
df['logalpha'] = ops2['log'](df.alpha)
df['logi'] = ops2['log'](df.i)
df['logz'] = ops2['log'](df.z)
df['logg'] = ops2['log'](df.g)
df['logr'] = ops2['log'](df.r)
df['logplate'] = ops2['log'](df.plate)
df['logred2'] = ops2['logadd'](df.redshift, df.logred)
df['z_div_g'] = ops2['div'](df.z, df.g)
df['ig'] = ops2['mul'](df.i, df.g)
df['redbool'] = (df.redshift < 0.15).astype(float)
df['redbool2'] = (df.redshift > 0.8).astype(float)
df['atan.logg'] = ops2['atan'](df.logg)
df['zbool'] = (df.z < 19.5).astype(float)
df['gbool'] = (df.g < 23.1).astype(float)
df['atan_alpha'] = ops2['atan'](df.alpha)

# Using Scaler to create a new feature
scaler = MinMaxScaler(feature_range=(0.00001, 1+0.00001))
df['redshift_scaled']= np.log(scaler.fit_transform(np.array(df['redshift']).reshape(-1, 1)))

# use the QuantileTrasformer which helped get a few more points.
trans = QuantileTransformer(n_quantiles=100, output_distribution='normal',random_state=RANDOM_STATE)
df['q_redshift'] = trans.fit_transform(np.array(df['redshift_scaled']).reshape(-1,1))
df['q_g'] = trans.fit_transform(np.array(df['g']).reshape(-1,1))
df['q_i'] = trans.fit_transform(np.array(df['i']).reshape(-1,1))
df['q_r'] = trans.fit_transform(np.array(df['r']).reshape(-1,1))
df['q_z'] = trans.fit_transform(np.array(df['z']).reshape(-1,1))
df['q_plate'] = trans.fit_transform(np.array(df['plate']).reshape(-1,1))
df['q_alpha'] = trans.fit_transform(np.array(df['alpha']).reshape(-1,1))
df['q_delta'] = trans.fit_transform(np.array(df['delta']).reshape(-1,1))

# Make a copy of the df, split it into the df with the 'Class' feature set
# And the testset for submission to Kaggle
df_orig= deepcopy(df)
df = deepcopy(df_orig.query('Class==Class'))
df_test = deepcopy(df_orig.query('Class != Class').drop('Class', axis=1))

# Do the split. Note it's shuffled before the split is done.
train,test = shuffle_and_split_data(df,0.2)

# Create the train and the train_y (targets)
train, train_y = train.drop('Class',axis=1),train.Class
# Create the test and test_y
test,test_y = test.drop('Class',axis=1),test.Class

# Make the pipeline
m = make_pipeline(PolynomialFeatures(interaction_only=True,degree=2),VarianceThreshold(),SelectKBest(k='all'),LDA(solver='svd',tol=0.00001))

# Cross Validation
#scores = cross_val_score(m,train,train_y.values.ravel(),n_jobs=-1)
#print("%0.5f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Perform the fit, make the predictions and create the CSV
print(m.fit(train, train_y.values.ravel()).score(test, test_y.values.ravel()))
pY = pd.DataFrame(m.predict(df_test), index=range(1,len(df_test)+1), columns=['Class']) # ensure that labels and observations are in corresponding order
df.Class.fillna('unknown').value_counts() # distribution of all train labels
pY.value_counts() # distribution of predicted labels
ToCSV(pY, 'MySubmission')

0.97003125
CPU times: user 51.3 s, sys: 4.35 s, total: 55.7 s
Wall time: 52.6 s


# **References:**

[1]. Hands-On Machine Learning (HOML) with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems (2nd edition). Géron, A. (2019). O'Reilly Media ISBN-13: 978-1492032649

[2]. Scikit-learn.org, 'API Reference — scikit-learn 1.2.2 documentation.' [Online]. Available: https://scikit-learn.org/stable/modules/classes.html

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 53 sec


## 💡**Starter Ideas**

1. Tune model hyperparameters
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest missclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support, or can the pattern be linearized via transformations?
1. How are date/categorical features treated by the model? Is there a [better way](https://www.google.com/search?q=ways+to+encode+categorical+data) to encode these (perhaps, ordinal) features?
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?
1. Learn astronomy domain and features: [🎦](https://www.youtube.com/results?search_query=Quasi-star), [quasi-star](https://en.wikipedia.org/wiki/Quasi-star), [star](https://en.wikipedia.org/wiki/Star), [galaxy](https://en.wikipedia.org/wiki/Galaxy), [📃](https://arxiv.org/abs/2112.02026)
